# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set_style("whitegrid")
import yfinance as yf
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def d(*obj):
    for o in obj:
        display(o)

def p(*obj):
    for o in obj:
        print(o)

def _np(ls):
    return np.array(ls)

def r(f, unit=3):
    return round(f, unit)

/Users/kyro_m3pro/Desktop/Backtesting/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Get Data

In [2]:
def str_to_float(x:str):
    if type(x) == str:
        return float(x.replace(',' ,''))
    else:
        return x
    
vixf = pd.read_csv('Data/VIXF.csv').set_index('Date')[::-1]
vixf.index = pd.to_datetime(pd.to_datetime(vixf.index).strftime('%Y-%m-%d'))

cols_to_convert = ["Open", "High", "Low", "Price"]
for col in cols_to_convert:
    vixf[col] = vixf[col].apply(str_to_float)

In [3]:
spx = pd.read_csv('Data/SPX.csv', index_col='Date')
spx.index = pd.to_datetime(spx.index)

vix = pd.read_csv('Data/VIX.csv', index_col='Date')
vix.index = pd.to_datetime(vix.index)

In [4]:
df = pd.concat([spx['Adj Close'].rename('SPX'),
                vix['Adj Close'].rename('VIX'),
                vixf['Price'].rename('VIXF')], axis=1)
df = df.dropna()
spread = df['VIXF'] - df['VIX']

# Transform

In [5]:
def smoothed_rolling_z(raw, window_sma: int, window_z: int):
    smoothed = raw.rolling(window_sma).mean()
    rolling_z = (smoothed - smoothed.rolling(window_z).mean()) / smoothed.rolling(window_z).std()
    return rolling_z

In [6]:
def calc_future_expected_return(_return, n):
    return _return[::-1].rolling(n).mean()[::-1].shift(-1)

# Optimization

In [7]:
import sys
sys.path.append('../../qwok/')
from visualization import Visualization as Vis
from signal_oscillator import Price_Oscillator_Signal

In [10]:
minima_res_arr = []

for window_sma in range(1, 50, 5):
    print(window_sma, end=" ")
    for window_z in range(20, 50, 5):
        rolling_z = smoothed_rolling_z(spread, window_sma, window_z)
        for holding_n in range(5, 31):
            for order in range(1, 6):
                obj_singals = Price_Oscillator_Signal(df['SPX'], rolling_z)
                signals_minima = obj_singals.getSignals_OscillatorExtrema(order)
                
                obj_bt = Vis(df['SPX'], signals_minima, holding_n=holding_n, ta=rolling_z)
                res_dict = {
                        'obj_bt': obj_bt,
                        'window_sma': window_sma,
                        'window_z': window_z,
                        'order': order,
                        'holding_n': holding_n,
                }
                for metrics, value in obj_bt.stat.items():
                    res_dict[metrics] = value
                minima_res_arr.append(res_dict)

1 6 11 16 21 26 31 36 41 46 

In [11]:
minima_bt = pd.DataFrame(minima_res_arr)

In [ ]:
if 1==0:
    minima_bt.to_excel('Result (Minima) 5.0.xlsx')

## Heatmap

In [ ]:
ax = sns.heatmap(minima_bt.pivot_table(index='window_sma' , columns='window_z', values='sharpe_ratio', aggfunc='mean'), cmap='RdBu')
plt.show()

In [ ]:
ax = sns.heatmap(minima_bt.pivot_table(index='order', columns='holding_n', values='sharpe_ratio', aggfunc='mean'), cmap='RdBu')
plt.show()

# Review

In [ ]:
idx = 2073 #2073
this_obj = minima_bt.loc[idx, 'obj_bt']

In [ ]:
this_obj.plot_general(config_position={'plot':False})

In [ ]:
this_obj.plot_equityCurve()

In [ ]:
this_obj.plot_signal(plot_extend_position=False)